## <font color=red>Литература</font>

1. Лекции «Алгоритмы: построение, анализ и реализация на языке программирования Си» - Ворожцов А.В., Винокуров Н.А.. Выложена в канале **#edu_materials**

# <font color=blue>Деревья. Продолжение 2</font>

# <font color=blue>Балансировка двоичного дерева</font>

Количество операций, необходимое для того, чтобы найти элемент в дереве, связано с высотой дерева $H$. Поиск элемента требует $O(H)$ операций. Если дерево случайное, то его высота может быть равна количеству узлов дерева $N$, а в лучшем случае высота двоичного дерева - $\left\lceil\log N\right\rceil$. Чтобы высота дерева в худшем случае была логарифмической, применяют **балансировку**.

**Методы балансировки деревьев поиска**  — это алгоритмы выполнения операций добавления и удаления записей (insert и del), которые гарантируют, что при любой последовательности выполнения запросов высота $H$ дерева поиска будет ограничена сверху линейной функцией от логарифма числа N хранимых записей:

$$H < A \cdot log_2 N + B$$

где $A$ и $B$ - фиксированные константы.

**Теорема**

Если в двоичном дереве c $N$ узлами выполнено хотя бы одно из следующих условий:

а) для любого узла число узлов в правом и левом поддереве $N_r$ , $N_l$ отличаются не более чем на $1$:
  $$N_r \le N_l + 1,\quad\quad N_l \le N_r +1$$
  
б) для любого узла число узлов в правом и левом поддереве $N_r$ , $N_l$ удовлетворяют условиям
  $$N_r \le 2 N_l + 1,\quad\quad N_l \le 2 N_r +1$$
  
а) для любого узла высота правого и левого поддеревьев $H_r$ , $H_l$ отличаются не более чем на $1$:
  $$H_r \le H_l + 1,\quad\quad H_l \le H_r +1$$
  
то высота дерева не превосходит $A \cdot log_2 N + B$, где $A$ и $B$ – некоторые положительные
константы, не зависящие от $N$.

**Доказательства** приведены в [1]

**Идально сбалансированное дерево** — это двоичное дерево поиска, для которого с помощью специальных алгоритмов поддерживается свойство (а).

**АВЛ-дерево** — это двоичное дерево поиска, для которого с помощью специальных алгоритмов поддерживается свойство (в).

Высота деревьев поиска, удовлетворяющих условиям (а), (б), (в) ограничена сверху логарифмической функцией от числа узлов дерева, а именно:

а) $H \le \log_2 N$

б) $H \le 1.70951 \log_2 N + 1$

в) $H \le 1.4404 \log_2 N + 1$

## <font color=green>АВЛ-дерево</font>

Названо так в честь математиков Адельского-Вельского и Ландиса.

Пусть высота правого поддерева равна $(k+2)$, а высота левого поддерева - $k$.

<img src="images/tree_rotations.png" alt="Drawing" style="width: 800px">

### Упражнение 1. Правое вращение

Добавьте в узлы ссылки на родителей. Реализуйте правое вращение.

In [21]:
class BSTNode:
    def __init__(self, key, value, left=None, right=None, parent=None):
        self.value = value
        self.key = key
        self.left = left
        self.right = right
        self.parent = parent
        
    def __str__(self):
        t = (
            repr(self.key),
            repr(self.value),
            'None' if self.left is None else repr(self.left.key),
            'None' if self.right is None else repr(self.right.key),
            'None' if self.parent is None else repr(self.parent.key)
        )
        fs = '(key={}, value={}, left_key={}, right_key={}, parent_key={})'
        return self.__class__.__name__ + fs.format(*t)
    
    def is_left_child(self):
        if self.parent is None:
            return False
        if self is self.parent.left:
            return True
        return False
    
    def is_right_child(self):
        if self.parent is None:
            return False
        if self is self.parent.right:
            return True
        return False
    
    def get_attribute_values_string(self, attributes):
        values = [getattr(self, attr) for attr in attributes]
        attr_strings = ['{}={}'.format(attr, v) for attr, v in zip(attributes, values)]
        return ' '.join(attr_strings)
        

class BST:
    def __init__(self):
        self.root = None
        
    def _insert(self, key, value, root):
        if key == root.key:
            root.value = value
        elif key < root.key:
            if root.left is None:
                root.left = BSTNode(key, value, parent=root)
            else:
                self._insert(key, value, root.left)
        else:
            if root.right is None:
                root.right = BSTNode(key, value, parent=root)
            else:
                self._insert(key, value, root.right)
        
    def insert(self, key, value):
        if self.root is None:
            self.root = BSTNode(key, value)
        else:
            self._insert(key, value, self.root)
            
    def _find(self, key, root):
        if root is None:
            return None
        if key == root.key:
            return root.value
        elif key < root.key:
            return self._find(key, root.left)
        else:
            return self._find(key, root.right)
            
    def find(self, key):
        return self._find(key, self.root)
    
    def _traverse(self, root):
        if root is not None:
            self._traverse(root.left)
            print(root.key, root.value)
            self._traverse(root.right)
    
    def traverse(self):
        self._traverse(self.root)
        
    @staticmethod
    def _get_connection_lines_for_children(connections):
        connections = connections.copy()
        if len(connections) == 0:
            return connections
        if connections[-1] == '├':
            connections[-1] = '│'
        elif connections[-1] == '└':
            connections[-1] = ' '
        else:
            raise ValueError('no connection to current node')
        return connections
            
    @staticmethod
    def _print_connections(connections):
        connections_str = ''
        for c in connections:
            if c == ' ':
                connections_str += c*4
            elif c == '│':
                connections_str += c + ' '*3
            elif c in '├└':
                connections_str += c + '─'*2 + ' '
            else:
                raise ValueError("unknown character '{}' in connections list".format(c))
        print(connections_str, end='')
        
    def _prefix_traverse(self, root, connections, attributes):
        self._print_connections(connections)
        connections = self._get_connection_lines_for_children(connections)
        if root is not None:
            print(root.key, root.get_attribute_values_string(attributes))
            connections_right = connections.copy()
            connections_right.append('├')
            self._prefix_traverse(root.right, connections_right, attributes)
            connections_left = connections.copy()
            connections_left.append('└')
            self._prefix_traverse(root.left, connections_left, attributes)
        else:
            print('-')
        
    def prefix_traverse(self, attributes=()):
        self._prefix_traverse(self.root, [], attributes)
    
    def _rotate_left(self, node):
        r = node.right
        if r is None:
            raise ValueError("no right subtree")
        rl = r.left

        p = node.parent
        
        if p is None:
            self.root = r
        else:
            if node.is_left_child():
                p.left = r
            else:
                p.right = r
        
        r.parent = p
        r.left = node
        
        node.parent = r
        node.right = rl
        
        if rl is not None:
            rl.parent = node
         
    def _rotate_right(self, node):
        left = node.left
        if left is None:
            raise ValueError("no left subtree")
        lr = left.right

        p = node.parent
        
        if p is None:
            self.root = left
        else:
            if node.is_left_child():
                p.left = left
            else:
                p.right = left
        
        left.parent = p
        left.right = node
        
        node.parent = left
        node.left = lr
        
        if lr is not None:
            lr.parent = node

## <font color=green>Пересчет балансов при вращениях</font>

<img src="images/tree_rotations.png" alt="Drawing" style="width: 800px">

Для контроля над сбалансированностью дерева в каждый узел добавляется специальный параметр, называемый балансом. Баланс равен разности высот правого и левого поддеревьев.

$$B = H_r - H_l$$

Расмотрим вопрос о пересчете балансов при левом вращении.

Узлы будут обозначаться так же, как и на рисунках выше. Высоты поддеревьев будут обозначаться буквой 'H', а балансы узлов буквой 'B', например $H_{rr}$, $B_n$. Высоты и балансы после вращения обозначаются шляпкой: $\hat{H_n}$, $\hat{B_r}$. **Обратите внимание, что названия узлов при вращении не меняются: позиция узла изменяется, но имя остается прежним.**

Для дерева на левом рисунке верны соотношения.

$$B_n = H_r - H_l$$
$$B_r = H_{rr} - H_{rl}$$
$$\hat{B_n} = H_{rl} - H_l$$
$$\hat{B_r} = H_{rr} - \hat{H_n}$$
$$H_r = \max(H_{rl}, H_{rr}) + 1$$
$$\hat{H_n} = \max(H_l, H_{rl}) + 1$$

При **левом вращении** изменятся балансы узлов $n$ и $r$. Вычислим их.


$$\hat{B_n} - B_n = H_{rl} - H_l - \left[\left(\max\left(H_{rl}, H_{rr}\right) + 1\right) - H_l\right]$$
$$\hat{B_n} - B_n = H_{rl} - \left(\max\left(H_{rl}, H_{rr}\right) + 1\right)$$
$$\hat{B_n} - B_n =  - \left(\max\left(H_{rl}, H_{rr}\right) -H_{rl} + 1\right)$$
$$\hat{B_n} - B_n =  - \left(\max\left(H_{rl}-H_{rl}, H_{rr}-H_{rl}\right)  + 1\right)$$
$$\hat{B_n} - B_n =  - \left(\max\left(0, B_r\right)  + 1\right)$$
$$\hat{B_n} = B_n - \max\left(B_r, 0\right)  - 1$$

Теперь вычисляем $\hat{B_r}$.

$$\hat{B_r} = H_{rr} - \hat{H_n}$$
$$\hat{B_r} = H_{rr} - \left[\max\left(H_l, H_{rl}\right) + 1\right]$$
$$\hat{B_r} = - \left[\max\left(H_l, H_{rl}\right) - H_{rr} + 1\right]$$
$$\hat{B_r} = - \left[\max\left(H_l - H_{rr}, H_{rl} - H_{rr}\right) + 1\right]$$
$$\hat{B_r} = - \left[\max\left(H_l - H_{rl} + H_{rl} - H_{rr}, -B_r\right) + 1\right]$$
$$\hat{B_r} = - \left[\max\left(-\hat{B_n}-B_r, -B_r\right) + 1\right]$$
$$\hat{B_r} = - \left[\max\left(-\hat{B_n}, 0\right) -B_r + 1\right]$$
$$\hat{B_r} = B_r + \min\left(\hat{B_n}, 0\right) - 1$$

Чтобы не проделывать ту же работу для вычисления новых значений балансов для **правого вращения**, воспользуемся симметрией процесса балансировки. Если 

- заменить знак всех балансов на противоположный,

- поменять направление балансировки (правое поддерево меньше корня, левое - больше) (баланс по прежнему вычисляется согласно $H_r - H_l$) и 

- отразить дерево относительно оси, проходящей через корень, 

то получится АВЛ-дерево, ожидающее левого поворота. В результате такого поворота отраженное дерево будет меняться зеркально с оригинальным деревом, причем балансы у деревьев будут отличаться только знаком.

Выкладки сверху также работают и для левого поворота в зеркальном дереве, поэтому для правого поворота в оригинальном дереве имеем

$$- \hat{B_n} = - B_n - \max\left(- B_r, 0\right)  - 1$$
$$ \hat{B_n} =  B_n + \max\left(- B_r, 0\right)  + 1$$
$$ \hat{B_n} =  B_n - \min\left(B_r, 0\right)  + 1$$

А для $\hat{B_r}$

$$- \hat{B_r} = - B_r + \min\left(- \hat{B_n}, 0\right) - 1$$
$$\hat{B_r} = B_r - \min\left(- \hat{B_n}, 0\right) + 1$$
$$\hat{B_r} = B_r + \max\left( \hat{B_n}, 0\right) + 1$$

## <font color=green>Вставка узла в АВЛ-дерево</font>

Процедура вставки состоит из двух этапов:

1. обычная вставка в двоичное дерево,

2. ретрейсинг, в процессе которого поправлялются значения балансов и осуществляется балансировка, где это необходимо.

Для балансировки при вставке достаточно одного 'большого' или одного 'малого' поворота, так как они уменьшают высоту вращаемого поддерева на 1.

In [63]:
class AVLNode(BSTNode):
    def __init__(self, key, value, left=None, right=None, parent=None, balance=0):
        super().__init__(key, value, left=left, right=right, parent=parent)
        self.balance = balance
        
        
class AVLTree(BST):                
    def _get_change_of_height_for_left_rotation(self, node_balance, right_balance):
#         print(node_balance, right_balance)
        l_height = 0
        r_height = node_balance
        rr_height = r_height - 1 if right_balance > 0 else r_height - 1 + right_balance
        rl_height = r_height - 1 if right_balance < 0 else r_height - 1 - right_balance
        old_height = max(l_height, r_height) + 1
        new_height = max(l_height+2, rl_height+2, rr_height+1)
        return new_height - old_height
    
    def _rotate_left(self, node):
        height_change = self._get_change_of_height_for_left_rotation(node.balance, node.right.balance)
        node.balance = node.balance - max(node.right.balance, 0) - 1
        node.right.balance = node.right.balance + min(node.balance, 0) - 1
        super()._rotate_left(node)
        return height_change
        
    def _rotate_right(self, node):
        height_change = self._get_change_of_height_for_left_rotation(-node.balance, -node.left.balance)
        node.balance = node.balance - min(node.left.balance, 0) + 1
        node.left.balance = node.left.balance + max(node.balance, 0) + 1
        super()._rotate_right(node)
        return height_change
    
    def _balance_unbalanced_node(self, node):
        assert node.balance > 1 or node.balance < -1, "node does not need balancing\n{}".format(node)
        if node.balance == -2:
            left = node.left
            assert left.balance != 0, \
                "tree was unbalanced before operation\nnode:\n{}\nleft child:\n{}".format(node, left)
            assert -3 < left.balance < 3, \
                "unsupported balance value in node\n{}".format(left)
            if left.balance == -1:
                height_change = self._rotate_right(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
            else:
                height_change = self._rotate_left(left)
                assert height_change == 0, \
                    "height change is not zero on first stage of big rotation\n" \
                    "height_change={}".format(height_change)
                height_change = self._rotate_right(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
        else:
            right = node.right
            assert right.balance != 0, \
                "tree was unbalanced before operation\nnode:\n{}\right child:\n{}".format(node, right)
            assert -3 < right.balance < 3, \
                "unsupported balance value in node\n{}".format(right)
            if right.balance == 1:
                height_change = self._rotate_left(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
            else:
                height_change = self._rotate_right(right)
                self._prefix_traverse(right.parent, [], ['balance'])
                assert height_change == 0, \
                    "height change is not zero on first stage of big rotation\n" \
                    "height_change={}".format(height_change)
                height_change = self._rotate_left(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
    
    def _retrace_insert_my(self, node):
        parent = node.parent
        if parent is None:
            return
        assert -2 < parent.balance < 2, "tree was unbalanced before insertion\nbalance={}".format(parent.balance)
        if node.is_left_child():
            parent.balance -= 1
            if parent.balance == 0:
                return
            if parent.balance == -1:
                self._retrace_insert_my(parent)
            else:
                self._balance_unbalanced_node(parent)
        else:
            parent.balance += 1
            if parent.balance == 0:
                return
            if parent.balance == 1:
                self._retrace_insert_my(parent)
            else:
                self._balance_unbalanced_node(parent)
                
    
    def _insert(self, key, value, root):
        if key == root.key:
            root.value = value
        elif key < root.key:
            if root.left is None:
                root.left = AVLNode(key, value, parent=root)
                self._retrace_insert_my(root.left)
            else:
                self._insert(key, value, root.left)
        else:
            if root.right is None:
                root.right = AVLNode(key, value, parent=root)
                self._retrace_insert_my(root.right)
            else:
                self._insert(key, value, root.right)
    
    def insert(self, key, value):
        if self.root is None:
            self.root = AVLNode(key, value)
        else:
            self._insert(key, value, self.root)

In [ ]:
import random

avl = AVLTree()
for i in [0, 1, 4, 3, 5, 2]:
    print('*********')
    print('INSERTING', i)
    avl.insert(i, str(i))
    avl.prefix_traverse(['balance'])
print('#######################################')

for i in range(100):
    i = random.randint(0, 1000)
    
    print('*********')
    print('INSERTING', i)
    avl.insert(i, str(i))
    avl.prefix_traverse(['balance'])
    
avl.prefix_traverse()

avl._rotate_right(avl.root)    
print('*************************')
avl.prefix_traverse()

## <font color=green>Вставка узла в АВЛ-дерево. Удаление узла из АВЛ-дерева</font>

Алгоритмы состоят из двух этапов.

1. Обычные вставка и удаление, используемые в двоичных деревьях поиска

2. Ретрейсинг. Проходим по дерево снизу вверх, балансируя дерево.
  
  - Для выполнения этой операции в каждый узел добавляется дополнительный параметр - баланс, который равен разности высот правого и левого дерева.
  
  - В процессе движения вверх исправляется баланс и дерево балансируется. Для балансировки достаточно выполнить одну операцию малого поворота (Rotate-Left, Rotate-Right) или одну операцию большого поворота (Rotate-Right-Left, Rotate-Left-Right)

### Упражнение 2. Вставка в АВЛ-дерево